**Etude de Marché - Exportation de poulet à l'international**

    Partie 1 - Nettoyage et préparation des données.

**DATA SOURCES**

http://www.fao.org/faostat/en/#data

***Population par pays en 2013 et 2017***
- Annual population > Population Est. & Proj > Total Both sexes > 2013, 2017

***Bilan alimentaire par pays en 2013***
- Food Balance Sheets > vegetal products, animal products > Food supply quantity, food supply, protein supply quantity > 2013

***Situation economique des pays en 2013***
- Suite of Food Security Indicators > Gross domestic product per capita, Political stability > 2013

# Tools

**Librairies**

In [50]:
# System
from pathlib import Path 

# Basic
import numpy as np
import pandas as pd

**Data**

In [51]:
# Dataset sur les populations annuelles en 2013 et 2017
data_pop = Path.cwd() / "data" / "raw" / "pop2.csv" 

# Dataset sur la disponibilité alimentaire des pays
data_anim = Path.cwd() / "data" / "raw" / "anim.csv" # produits animaliers
data_vg = Path.cwd() / "data" / "raw" / "vg.csv" # produits végétaux

# Dataset sur l'économie locale des pays
data_eco = Path.cwd() / "data" / "raw" / "eco.csv"

# Data Population

*Table présentant les populations par pays en 2010 et 2013. Il existe des données disponibles après 2013, mais pour comparer de façon pertinente nos données provenant de différentes tables, on ne travaillera pas au-delà de 2013.*

## Nomenclature & Formatage colonnes

In [52]:
# Affichage des données

pop = pd.read_csv(data_pop)
pop.head()

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,OA,Annual population,2,Afghanistan,511,Total Population - Both sexes,3010,Population - Est. & Proj.,2013,2013,1000 persons,31731.688,X,International reliable sources,"UNDESA, Population Division – World Population..."
1,OA,Annual population,2,Afghanistan,511,Total Population - Both sexes,3010,Population - Est. & Proj.,2017,2017,1000 persons,35530.081,X,International reliable sources,"UNDESA, Population Division – World Population..."
2,OA,Annual population,3,Albania,511,Total Population - Both sexes,3010,Population - Est. & Proj.,2013,2013,1000 persons,2918.978,X,International reliable sources,"UNDESA, Population Division – World Population..."
3,OA,Annual population,3,Albania,511,Total Population - Both sexes,3010,Population - Est. & Proj.,2017,2017,1000 persons,2930.187,X,International reliable sources,"UNDESA, Population Division – World Population..."
4,OA,Annual population,4,Algeria,511,Total Population - Both sexes,3010,Population - Est. & Proj.,2013,2013,1000 persons,38338.562,X,International reliable sources,"UNDESA, Population Division – World Population..."


In [53]:
# Sélection des colonnes

pop = pop.loc[:,['Area Code', 'Area','Year', 'Value']]


# Renommage des colonnes

pop = pop.rename(index=str, columns={"Area Code":'Code_c',"Area": "Country", 'Value': 'Population'}) 


# Formatage de la colonne population en unité de personnes

pop['Population'] = (pop['Population'] * 1000).round(0) 

pop.head()

,Code_c,Country,Year,Population
0,2,Afghanistan,2013,31731688.0
1,2,Afghanistan,2017,35530081.0
2,3,Albania,2013,2918978.0
3,3,Albania,2017,2930187.0
4,4,Algeria,2013,38338562.0


In [54]:
# Tableau croisé dynamique sur data de population par années

pop = pop.pivot_table(

    index=['Code_c',"Country"],

    columns = ["Year"], values=["Population"]).reset_index()

# Renommage final 

pop.columns = ['Code_c',"Country",'pop2013','pop2017']

pop.head()


,Code_c,Country,pop2013,pop2017
0,1,Armenia,2893509.0,2930450.0
1,2,Afghanistan,31731688.0,35530081.0
2,3,Albania,2918978.0,2930187.0
3,4,Algeria,38338562.0,41318142.0
4,5,American Samoa,55307.0,55641.0


## Valeurs manquantes

In [55]:
# Combien de valeurs manquantes dans le dataframe ?

pop.isnull().sum()

Code_c     0
Country    0
pop2013    0
pop2017    0
dtype: int64

## Création de colonnes

In [56]:
# Création colonne 'Variation population'

pop['Var_pop (%)'] = ((pop['pop2017']-pop['pop2013'])/pop['pop2017']).map(lambda x : "%e"%x)
pop.head()

,Code_c,Country,pop2013,pop2017,Var_pop (%)
0,1,Armenia,2893509.0,2930450.0,1.260591e-02
1,2,Afghanistan,31731688.0,35530081.0,1.069064e-01
2,3,Albania,2918978.0,2930187.0,3.825353e-03
3,4,Algeria,38338562.0,41318142.0,7.211312e-02
4,5,American Samoa,55307.0,55641.0,6.002768e-03


## Valeurs incohérentes

In [57]:
# Population mondiale

pop['pop2013'].sum()

7213211612.0

In [58]:
pop['pop2017'].sum()

7550036044.0

<div class="alert alert-block alert-info">
<b>Tip:</b> Résultats cohérent aux données officielles.
    
http://www.worldometers.info/fr/


https://www.ined.fr/fichier/s_rubrique/18709/population_societes_2013_503_population_monde.fr.pdf

</div>

# Data Alimentation

*Table présentant la disponibilité alimentaire des pays en 2013. Pas de données disponibles après 2013.*

## Dataset originaux

In [59]:
# Affichage des données sur produits animaliers

meat = pd.read_csv(data_anim)
meat.head()

,Domain Code,Domain,Country Code,Country,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,FBS,Food Balance Sheets,2,Afghanistan,645,Food supply quantity (kg/capita/yr),2731,Bovine Meat,2013,2013,kg,4.59,Fc,Calculated data
1,FBS,Food Balance Sheets,2,Afghanistan,664,Food supply (kcal/capita/day),2731,Bovine Meat,2013,2013,kcal/capita/day,27.00,Fc,Calculated data
2,FBS,Food Balance Sheets,2,Afghanistan,674,Protein supply quantity (g/capita/day),2731,Bovine Meat,2013,2013,g/capita/day,1.89,Fc,Calculated data
3,FBS,Food Balance Sheets,2,Afghanistan,645,Food supply quantity (kg/capita/yr),2732,Mutton & Goat Meat,2013,2013,kg,4.92,Fc,Calculated data
4,FBS,Food Balance Sheets,2,Afghanistan,664,Food supply (kcal/capita/day),2732,Mutton & Goat Meat,2013,2013,kcal/capita/day,31.00,Fc,Calculated data


In [60]:
# Affichage des données sur produits végétaux

vg = pd.read_csv(data_vg)
vg.head()

,Domain Code,Domain,Country Code,Country,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,FBS,Food Balance Sheets,2,Afghanistan,645,Food supply quantity (kg/capita/yr),2511,Wheat and products,2013,2013,kg,160.23,Fc,Calculated data
1,FBS,Food Balance Sheets,2,Afghanistan,664,Food supply (kcal/capita/day),2511,Wheat and products,2013,2013,kcal/capita/day,1369.00,Fc,Calculated data
2,FBS,Food Balance Sheets,2,Afghanistan,674,Protein supply quantity (g/capita/day),2511,Wheat and products,2013,2013,g/capita/day,36.91,Fc,Calculated data
3,FBS,Food Balance Sheets,2,Afghanistan,645,Food supply quantity (kg/capita/yr),2805,Rice (Milled Equivalent),2013,2013,kg,13.82,Fc,Calculated data
4,FBS,Food Balance Sheets,2,Afghanistan,664,Food supply (kcal/capita/day),2805,Rice (Milled Equivalent),2013,2013,kcal/capita/day,141.00,Fc,Calculated data


In [61]:
# Création de colonnes 'origine' dans les 2 dataset

meat["Origin"] = "animal"

vg["Origin"] = "vegetal"

In [62]:
# On regroupe vg et meat en un unique dataframe, via une union

food = meat.append(vg)

# Suppression des 2 dataset vg et meat

del meat, vg

In [63]:
food.head()

,Domain Code,Domain,Country Code,Country,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Origin
0,FBS,Food Balance Sheets,2,Afghanistan,645,Food supply quantity (kg/capita/yr),2731,Bovine Meat,2013,2013,kg,4.59,Fc,Calculated data,animal
1,FBS,Food Balance Sheets,2,Afghanistan,664,Food supply (kcal/capita/day),2731,Bovine Meat,2013,2013,kcal/capita/day,27.00,Fc,Calculated data,animal
2,FBS,Food Balance Sheets,2,Afghanistan,674,Protein supply quantity (g/capita/day),2731,Bovine Meat,2013,2013,g/capita/day,1.89,Fc,Calculated data,animal
3,FBS,Food Balance Sheets,2,Afghanistan,645,Food supply quantity (kg/capita/yr),2732,Mutton & Goat Meat,2013,2013,kg,4.92,Fc,Calculated data,animal
4,FBS,Food Balance Sheets,2,Afghanistan,664,Food supply (kcal/capita/day),2732,Mutton & Goat Meat,2013,2013,kcal/capita/day,31.00,Fc,Calculated data,animal


## Nomenclature et Formatage 

In [64]:
# Renommage des colonnes

food = food.rename(index=str, columns={"Country Code":'Code_c'}) 

# Création tableau croisé dynamique

food = food.pivot_table(

    index=['Code_c',"Country","Year", "Origin","Item"],

    columns = ["Element"], values=["Value"], aggfunc=sum)

food.head()

Value  \
Element                                            Food supply (kcal/capita/day)   
Code_c Country Year Origin Item                                                    
1      Armenia 2013 animal Aquatic Animals, Others                           0.0   
                           Aquatic Plants                                    0.0   
                           Bovine Meat                                     114.0   
                           Butter, Ghee                                     35.0   
                           Cephalopods                                       0.0   

                                                                                        \
Element                                            Food supply quantity (kg/capita/yr)   
Code_c Country Year Origin Item                                                          
1      Armenia 2013 animal Aquatic Animals, Others                                0.00   
                           Aquatic Plants                                         0.00   
                           Bovine Meat                                           19.66   
                           Butter, Ghee                                           1.76   
                           Cephalopods                                            0.00   

                                                                                           
Element                                            Protein supply quantity (g/capita/day)  
Code_c Country Year Origin Item                                                            
1      Armenia 2013 animal Aquatic Animals, Others                                   0.00  
                           Aquatic Plants                                            0.00  
                           Bovine Meat                                               7.97  
                           Butter, Ghee                                              0.04  
                           Cephalopods                                               0.00

In [65]:
# Renommage colonnes

food.columns = ["FS (kcal/pers/d)",'FS (kg/pers/d)', 'ProtS (g/pers/d)'] 

## Sélection des données

In [66]:
# Aggrégation par pays / origine / années 

food = food.reset_index()

food = food.drop(columns='Year') # Suppression de la colonne Year (unique)

food = food.groupby(['Code_c','Country','Origin']).sum().reset_index()

food.head()

,Code_c,Country,Origin,FS (kcal/pers/d),FS (kg/pers/d),ProtS (g/pers/d)
0,1,Armenia,animal,784.0,280.99,43.24
1,1,Armenia,vegetal,2140.0,704.69,46.82
2,2,Afghanistan,animal,216.0,79.92,12.21
3,2,Afghanistan,vegetal,1871.0,271.49,46.05
4,3,Albania,animal,985.0,389.05,59.41


## Valeurs manquantes

In [67]:
# Combien de valeurs manquantes dans le dataframe ?

food.isnull().sum()

Code_c              0
Country             0
Origin              0
FS (kcal/pers/d)    0
FS (kg/pers/d)      0
ProtS (g/pers/d)    0
dtype: int64

## Création de colonnes

In [68]:
# Tableau croisé dynamique sur protéines

temp = food.pivot_table(

    index=['Code_c',"Country"],

    columns = ["Origin"], values=["ProtS (g/pers/d)"], aggfunc=sum)

temp.columns = ['ProtA (g/pers/d)', 'ProtVG (g/pers/d)'] # renommage des colonnes en fonction de l'origine Animal ou Vegetal

temp = temp.reset_index()

temp.head()

,Code_c,Country,ProtA (g/pers/d),ProtVG (g/pers/d)
0,1,Armenia,43.24,46.82
1,2,Afghanistan,12.21,46.05
2,3,Albania,59.41,51.96
3,4,Algeria,24.98,66.94
4,7,Angola,18.40,38.87


In [69]:
# Création colonnes Proportion de protéines d'origine animale / quantité totale de protéines

temp['%ProtA'] = (temp['ProtA (g/pers/d)'] / (temp['ProtA (g/pers/d)'] + temp['ProtVG (g/pers/d)'])).map(lambda x : "%e"%x)

temp.head()

,Code_c,Country,ProtA (g/pers/d),ProtVG (g/pers/d),%ProtA
0,1,Armenia,43.24,46.82,4.801244e-01
1,2,Afghanistan,12.21,46.05,2.095778e-01
2,3,Albania,59.41,51.96,5.334471e-01
3,4,Algeria,24.98,66.94,2.717581e-01
4,7,Angola,18.40,38.87,3.212851e-01


## Table Food intermédiaire 

In [70]:
# Aggrégation sur pays et années

food = food.groupby(['Code_c','Country']).sum().reset_index()

food.head()

,Code_c,Country,FS (kcal/pers/d),FS (kg/pers/d),ProtS (g/pers/d)
0,1,Armenia,2924.0,985.68,90.06
1,2,Afghanistan,2087.0,351.41,58.26
2,3,Albania,3188.0,1095.54,111.37
3,4,Algeria,3293.0,809.13,91.92
4,7,Angola,2474.0,701.46,57.27


In [71]:
# Jointure interne

food = pd.merge(food, temp[['Code_c','Country','%ProtA']], on = ['Code_c','Country'], how = 'outer')

food.head()

,Code_c,Country,FS (kcal/pers/d),FS (kg/pers/d),ProtS (g/pers/d),%ProtA
0,1,Armenia,2924.0,985.68,90.06,4.801244e-01
1,2,Afghanistan,2087.0,351.41,58.26,2.095778e-01
2,3,Albania,3188.0,1095.54,111.37,5.334471e-01
3,4,Algeria,3293.0,809.13,91.92,2.717581e-01
4,7,Angola,2474.0,701.46,57.27,3.212851e-01


<div class="alert alert-block alert-info">
Jointure externe pour garder toutes les données des 2 dataframes. </div>

# Data Economie

*Data sur le PIB / habitants et l'indice de stabilité des pays en 2013.*

## Nomenclature & Formatage colonnes

In [72]:
# Affichage des données sur les échanges commerciaux

eco = pd.read_csv(data_eco)
eco.head()

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,FS,Suite of Food Security Indicators,2,Afghanistan,6126,Value,22013,"Gross domestic product per capita, PPP, dissem...",2013,2013,I$,1848.70,X,International reliable sources
1,FS,Suite of Food Security Indicators,2,Afghanistan,6125,Value,21032,Political stability and absence of violence/te...,2013,2013,index,-2.52,X,International reliable sources
2,FS,Suite of Food Security Indicators,3,Albania,6126,Value,22013,"Gross domestic product per capita, PPP, dissem...",2013,2013,I$,10492.80,X,International reliable sources
3,FS,Suite of Food Security Indicators,3,Albania,6125,Value,21032,Political stability and absence of violence/te...,2013,2013,index,0.09,X,International reliable sources
4,FS,Suite of Food Security Indicators,4,Algeria,6126,Value,22013,"Gross domestic product per capita, PPP, dissem...",2013,2013,I$,13253.60,X,International reliable sources


In [73]:
# Renommage des colonnes
    
eco.columns = ["xx","xx2",'Code_c',"Country",'xx3','xxx'

    ,'xxx','Item','xx4',"Year","unit","value",'xx5','xx6']

In [74]:
# Création tableau croisé dynamique

eco = eco.pivot_table(

    index=['Code_c',"Country"],

    columns = ["Item"], values=["value"], aggfunc=sum)

eco.head()

value  \
Item                  Gross domestic product per capita, PPP, dissemination (constant 2011 international $)   
Code_c Country                                                                                                
1      Armenia                                                    7727.9                                      
2      Afghanistan                                                1848.7                                      
3      Albania                                                   10492.8                                      
4      Algeria                                                   13253.6                                      
5      American Samoa                                                NaN                                      

                                                                                     
Item                  Political stability and absence of violence/terrorism (index)  
Code_c Country                                                                       
1      Armenia                                                      0.11             
2      Afghanistan                                                 -2.52             
3      Albania                                                      0.09             
4      Algeria                                                     -1.20             
5      American Samoa                                               0.93

In [75]:
# Renommage des colonnes

eco.columns = ['PIB$/hab', 'Ix_stab'] 

eco = eco.reset_index()
eco.head()

,Code_c,Country,PIB$/hab,Ix_stab
0,1,Armenia,7727.9,0.11
1,2,Afghanistan,1848.7,-2.52
2,3,Albania,10492.8,0.09
3,4,Algeria,13253.6,-1.20
4,5,American Samoa,NaN,0.93


## Valeurs manquantes

In [76]:
# Affichage des valeurs manquantes sur PIB

eco.isnull().sum()

Code_c      0
Country     0
PIB$/hab    9
Ix_stab     1
dtype: int64

In [77]:
eco.loc[eco['Ix_stab'].isnull()]

,Code_c,Country,PIB$/hab,Ix_stab
193,351,China,12206.8,NaN


In [78]:
eco.loc[eco['PIB$/hab'].isnull()]

,Code_c,Country,PIB$/hab,Ix_stab
4,5,American Samoa,NaN,0.93
5,6,Andorra,NaN,1.28
37,49,Cuba,NaN,0.37
53,72,Djibouti,NaN,-0.10
62,85,Greenland,NaN,1.93
87,116,Democratic People's Republic of Korea,NaN,-0.44
134,178,Eritrea,NaN,-0.77
152,201,Somalia,NaN,-2.76
160,212,Syrian Arab Republic,NaN,-2.68


<div class="alert alert-block alert-info">
<b>Tip:</b> 
    
- Pas de valeurs pour la Chine en termes de stabilité : pas vraiment surprenant.
- Il manque certaines valeurs pour le PIB/habitants : cela peut poser pb pour notre clustering par la suite. </div>

## Table Economie intermédiaire

In [79]:
# Table finale sur l'économie des pays

eco.head()

,Code_c,Country,PIB$/hab,Ix_stab
0,1,Armenia,7727.9,0.11
1,2,Afghanistan,1848.7,-2.52
2,3,Albania,10492.8,0.09
3,4,Algeria,13253.6,-1.20
4,5,American Samoa,NaN,0.93


# Table finale

## Jointures

In [80]:
# Jointure des dataframes population + food

data = pd.merge(food, pop[['Code_c','Country','pop2013','Var_pop (%)']], on=['Code_c', 'Country'], how='outer')

data.head()

,Code_c,Country,FS (kcal/pers/d),FS (kg/pers/d),ProtS (g/pers/d),%ProtA,pop2013,Var_pop (%)
0,1,Armenia,2924.0,985.68,90.06,4.801244e-01,2893509.0,1.260591e-02
1,2,Afghanistan,2087.0,351.41,58.26,2.095778e-01,31731688.0,1.069064e-01
2,3,Albania,3188.0,1095.54,111.37,5.334471e-01,2918978.0,3.825353e-03
3,4,Algeria,3293.0,809.13,91.92,2.717581e-01,38338562.0,7.211312e-02
4,7,Angola,2474.0,701.46,57.27,3.212851e-01,25998340.0,1.271095e-01


In [81]:
# Jointure du dataframe avec le dataframe Economie

data = pd.merge(data, eco , on=['Code_c','Country'], how='outer')

data.head()

,Code_c,Country,FS (kcal/pers/d),FS (kg/pers/d),ProtS (g/pers/d),%ProtA,pop2013,Var_pop (%),PIB$/hab,Ix_stab
0,1,Armenia,2924.0,985.68,90.06,4.801244e-01,2893509.0,1.260591e-02,7727.9,0.11
1,2,Afghanistan,2087.0,351.41,58.26,2.095778e-01,31731688.0,1.069064e-01,1848.7,-2.52
2,3,Albania,3188.0,1095.54,111.37,5.334471e-01,2918978.0,3.825353e-03,10492.8,0.09
3,4,Algeria,3293.0,809.13,91.92,2.717581e-01,38338562.0,7.211312e-02,13253.6,-1.20
4,7,Angola,2474.0,701.46,57.27,3.212851e-01,25998340.0,1.271095e-01,6188.0,-0.39


## Format

In [82]:
# Format final 

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 228 entries, 0 to 227
Data columns (total 10 columns):
Code_c              228 non-null int64
Country             228 non-null object
FS (kcal/pers/d)    171 non-null float64
FS (kg/pers/d)      171 non-null float64
ProtS (g/pers/d)    171 non-null float64
%ProtA              171 non-null object
pop2013             227 non-null float64
Var_pop (%)         227 non-null object
PIB$/hab            185 non-null float64
Ix_stab             193 non-null float64
dtypes: float64(6), int64(1), object(3)
memory usage: 19.6+ KB


In [83]:
# Reformatage des colonnes % proteines et variation de la population en %

data['%ProtA'] = data['%ProtA'].astype(float)

data['Var_pop (%)'] = data['Var_pop (%)'].astype(float)

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 228 entries, 0 to 227
Data columns (total 10 columns):
Code_c              228 non-null int64
Country             228 non-null object
FS (kcal/pers/d)    171 non-null float64
FS (kg/pers/d)      171 non-null float64
ProtS (g/pers/d)    171 non-null float64
%ProtA              171 non-null float64
pop2013             227 non-null float64
Var_pop (%)         227 non-null float64
PIB$/hab            185 non-null float64
Ix_stab             193 non-null float64
dtypes: float64(8), int64(1), object(1)
memory usage: 19.6+ KB


## NaN

### Table de base

In [84]:
# Valeurs manquantes

data.isnull().sum()

Code_c               0
Country              0
FS (kcal/pers/d)    57
FS (kg/pers/d)      57
ProtS (g/pers/d)    57
%ProtA              57
pop2013              1
Var_pop (%)          1
PIB$/hab            43
Ix_stab             35
dtype: int64

In [85]:
data.loc[data['FS (kcal/pers/d)'].isnull()].sort_values(by='pop2013', ascending = True)

,Code_c,Country,FS (kcal/pers/d),FS (kg/pers/d),ProtS (g/pers/d),%ProtA,pop2013,Var_pop (%),PIB$/hab,Ix_stab
188,94,Holy See,NaN,NaN,NaN,NaN,801.0,-0.011364,NaN,NaN
213,218,Tokelau,NaN,NaN,NaN,NaN,1187.0,0.086923,NaN,NaN
197,160,Niue,NaN,NaN,NaN,NaN,1623.0,-0.003090,NaN,NaN
182,65,Falkland Islands (Malvinas),NaN,NaN,NaN,NaN,2870.0,0.013746,NaN,NaN
205,187,"Saint Helena, Ascension and Tristan da Cunha",NaN,NaN,NaN,NaN,4081.0,-0.007903,NaN,NaN
194,142,Montserrat,NaN,NaN,NaN,NaN,5068.0,0.021055,NaN,NaN
206,190,Saint Pierre and Miquelon,NaN,NaN,NaN,NaN,6273.0,0.007437,NaN,NaN
196,148,Nauru,NaN,NaN,NaN,NaN,10790.0,0.050092,10078.4,1.05
216,227,Tuvalu,NaN,NaN,NaN,NaN,10819.0,0.033327,3143.1,1.28
219,243,Wallis and Futuna Islands,NaN,NaN,NaN,NaN,12551.0,-0.066083,NaN,NaN


<div class="alert alert-block alert-info">
<b>Tip:</b> 

- Les données manquantes sur les ressources alimentaires concernent en majorité des petits pays dont la population est inférieure à 1 millions d'habitants.

- Sur ces pays toutes les informations concernant les ressources alimentaires sont manquantes.

La question est de savoir comment les gérer ?

- Suppression.
- Imputation par la moyenne ou la médiane.
- Remplacement par zéro.

Vu qu'il nous manque toutes les informations concernant les ressources alimentaires qui sont cruciales pour notre analyse, je choisis de les supprimer.
</div>

In [86]:
data = data.loc[~data['FS (kcal/pers/d)'].isnull()]

In [87]:
# Focus sur les valeurs NAN sur PIB et index de stabilité

data.loc[(data['PIB$/hab'].isnull()) | (data['Ix_stab'].isnull())]

,Code_c,Country,FS (kcal/pers/d),FS (kg/pers/d),ProtS (g/pers/d),%ProtA,pop2013,Var_pop (%),PIB$/hab,Ix_stab
31,49,Cuba,3402.0,835.47,90.86,0.349769,1.141217e+07,0.006310,NaN,0.37
46,70,French Polynesia,2923.0,735.66,97.27,0.655906,2.735280e+05,0.033494,NaN,NaN
47,72,Djibouti,2609.0,454.14,65.62,0.229656,8.966880e+05,0.063007,NaN,-0.10
80,116,Democratic People's Republic of Korea,2093.0,458.93,54.99,0.183124,2.498598e+07,0.019811,NaN,-0.44
102,147,Namibia,2166.0,585.07,57.61,0.339351,2.316520e+06,0.085750,NaN,NaN
105,153,New Caledonia,2855.0,732.42,86.61,0.603972,2.618890e+05,0.052003,NaN,NaN
170,351,China,3112.0,897.11,98.05,0.404182,1.413857e+09,0.018925,12206.8,NaN


<div class="alert alert-block alert-info">
<b>Tip:</b>  Je garde la même logique que précédemment en ne conservant par les états dont la population est inférieure à 1 millions d'habitants. </div>

In [88]:
data = data.loc[data['pop2013']>1000000]

data.loc[(data['PIB$/hab'].isnull()) | (data['Ix_stab'].isnull())]

,Code_c,Country,FS (kcal/pers/d),FS (kg/pers/d),ProtS (g/pers/d),%ProtA,pop2013,Var_pop (%),PIB$/hab,Ix_stab
31,49,Cuba,3402.0,835.47,90.86,0.349769,1.141217e+07,0.006310,NaN,0.37
80,116,Democratic People's Republic of Korea,2093.0,458.93,54.99,0.183124,2.498598e+07,0.019811,NaN,-0.44
102,147,Namibia,2166.0,585.07,57.61,0.339351,2.316520e+06,0.085750,NaN,NaN
170,351,China,3112.0,897.11,98.05,0.404182,1.413857e+09,0.018925,12206.8,NaN


<div class="alert alert-block alert-info">
<b>Tip:</b>  Je ne garde pas Cuba et la Corée du Nord qui sont relativement 'fermés' donc peu favorables au dévelopement d'un marché. </div>

In [89]:
data = data.drop([31,80])

data.loc[(data['PIB$/hab'].isnull()) | (data['Ix_stab'].isnull())]

,Code_c,Country,FS (kcal/pers/d),FS (kg/pers/d),ProtS (g/pers/d),%ProtA,pop2013,Var_pop (%),PIB$/hab,Ix_stab
102,147,Namibia,2166.0,585.07,57.61,0.339351,2.316520e+06,0.085750,NaN,NaN
170,351,China,3112.0,897.11,98.05,0.404182,1.413857e+09,0.018925,12206.8,NaN


<div class="alert alert-block alert-info">
<b>Tip:</b>  Je remplace les valeurs restantes par les valeurs trouvées sur internet ou par 0 pour l'index de stabilité de la Chine. Ce dernier paramètre n'étant pas utilisé pour le clustering cela n'est pas génant. </div>

In [90]:
data.loc[102,'PIB$/hab'] = 9578

data[data.Country == 'Namibia']

,Code_c,Country,FS (kcal/pers/d),FS (kg/pers/d),ProtS (g/pers/d),%ProtA,pop2013,Var_pop (%),PIB$/hab,Ix_stab
102,147,Namibia,2166.0,585.07,57.61,0.339351,2316520.0,0.08575,9578.0,NaN


In [91]:
data = data.fillna(data.mean())

<div class="alert alert-block alert-info">
<b>Tip:</b> 
    
- Si je remplace les valeurs manquantes je vais biaiser le clustering. 
- Si je supprime ces valeurs pour quelques données manquantes je vais perdre des candidats potentiels avec des niveaux de population important. 

Par ailleurs, ces données présentent des valeurs extrêmes importantes qui vont influencer le clustering en créant des groupes de quelques individus.

**Je choisis de réaliser le clustering sur les données des ressources alimentaires (variable sélective) et je garde les autres données comme variables illustratives des groupes qui seront formés.**
Je peux donc imputer les valeurs manquantes par la moyenne sans craindre que cela biaise le clustering.</div>

In [92]:
data.isnull().sum()

Code_c              0
Country             0
FS (kcal/pers/d)    0
FS (kg/pers/d)      0
ProtS (g/pers/d)    0
%ProtA              0
pop2013             0
Var_pop (%)         0
PIB$/hab            0
Ix_stab             0
dtype: int64

## BackUp

In [93]:
# Sauvegarde des données nettoyées

pop.to_csv(Path.cwd() / "data" / "interim" / "pop_prep.csv", encoding='utf-8',index=False)

food.to_csv(Path.cwd() / "data" / "interim" / "food_prep.csv", encoding='utf-8',index=False)

eco.to_csv(Path.cwd() / "data" / "interim" / "eco_prep.csv", encoding='utf-8',index=False)

# Sauvegarde data finales

data.to_csv(Path.cwd() / "data" / "processed" / "data_prep.csv", encoding='utf-8',index=False)